In [7]:
path = 'drive/MyDrive/workspace/IR/A7'

In [8]:
cd $path

/content/drive/MyDrive/workspace/IR/A7


In [5]:
mv 'drive/MyDrive/Colab Notebooks/Assignment_7_202018005.ipynb' 'drive/MyDrive/workspace/IR/A7'

mv: cannot stat 'fdrive/MyDrive/Colab Notebooks/{fileName}.ipynb': No such file or directory


# A7

In [10]:
!touch utils.py

In [11]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [14]:
try:
    makeerror==1
except:
    !wget https://github.com/gravitational/teleconsole/releases/download/0.4.0/teleconsole-v0.4.0-linux-amd64.tar.gz
    !tar -xvf teleconsole-v0.4.0-linux-amd64.tar.gz
makeerror=False
import subprocess as sp
process = sp.Popen("./teleconsole",shell=True,stdin=sp.PIPE,stdout=sp.PIPE,stderr=sp.PIPE)
for i in range(6):
    print(process.stdout.readline().decode())
!apt install screen
!screen -d -m bash

Starting local SSH server on localhost...

Requesting a disposable SSH proxy on teleconsole.com for root...

Checking status of the SSH tunnel...



Your Teleconsole ID: 8ffde49d51b46e195d2d31603f613f56eb4dd9ba

WebUI for this session: https://teleconsole.com/s/8ffde49d51b46e195d2d31603f613f56eb4dd9ba

Reading package lists... Done
Building dependency tree       
Reading state information... Done
screen is already the newest version (4.6.2-1ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [ ]:
import pandas as pd
df = pd.read_csv('data/train.csv')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
dftest = pd.read_csv('data/test.csv')
dftest.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [ ]:
dftest.shape

(153164, 2)

In [ ]:
dftestlabels = pd.read_csv('data/test_labels.csv')
dftestlabels.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [ ]:
dftestlabels.shape

(153164, 7)

In [ ]:
columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
for col in columns:
  print(f'{col}:{sum(dftestlabels[col] == -1)}')

toxic:89186
severe_toxic:89186
obscene:89186
threat:89186
insult:89186
identity_hate:89186


There are 89186 test comments for which variables have -1 value. we need to remove those.

In [ ]:
dftlvalid = dftestlabels[dftestlabels['toxic'] != -1]
dftestvalid = dftest[dftestlabels['toxic'] != -1]


In [ ]:
# %%writefile setup.py

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.test.utils import datapath
from gensim import utils
from gensim.parsing.preprocessing import preprocess_string, preprocess_documents

class document:
  """
  custom class to represent comments
  """
  def __init__(self, id, text, *ls):
    """
    initialization of comments
    """
    self.id = id
    self.text = text
    r = iter(range(6))
    self.level = {threat:ls[next(r)] for threat in ['toxic', 'severe_toxic', 'obscene', 'threat', 
                                  'insult', 'identity_hate']}
  
  def read(self):
    """
    returns comment's text
    """
    return self.text




      

In [ ]:
#training comments
documents = [document(*list(df.iloc[i])) for i in range(df.shape[0])]

In [ ]:
#testing comments
testdocs = [document(dftestvalid.iloc[i, 0], dftestvalid.iloc[i, 1], *list(dftlvalid.iloc[ i, 1:])) for i in range(dftlvalid.shape[0])]

In [ ]:
def count():
  i = 0
  while True:
    yield i
    i += 1

In [ ]:
class vectorize:
  """
  custom class for tfidf and word2vec
  """
  def __init__(self, documents, testdocs):
    self.documents = documents
    self.testdocs = testdocs

  def iter(self):
    
    from gensim.parsing.preprocessing import preprocess_string, preprocess_documents
    for doc in self.documents:
      yield doc.read()
    for testd in self.testdocs:
      yield testd.read()


  def tfidfvec(self):
    """
    tfidf representation for training and testing comments
    """
    from sklearn.feature_extraction.text import TfidfVectorizer
    itr = self.iter()
    termdocMat = TfidfVectorizer(self.documents, token_pattern='[a-zA-Z]+', stop_words={'english'}).fit_transform(itr)
    self.tfidfdoc = termdocMat[:len(self.documents), :]
    c= count()
    self.tfidftestdoc = termdocMat[len(self.documents):,:]
    del termdocMat

  def word2vec(self, sentences):
    """
    word2vec representation for training and testing comments
    """
    from gensim.test.utils import datapath
    from gensim import utils
    from gensim.parsing.preprocessing import preprocess_string, preprocess_documents 

    import gensim.models

    skipmodel = gensim.models.Word2Vec(sentences=sentences, sg=1, min_count = 1, size=300, window=5, workers=2)

    
    docs = {file.id:file.read().lower().split() for file in self.documents}

    queries = {file.id:file.read().lower().split() for file in self.testdocs}

    for i in range(len(self.documents)):
      self.documents[i].w2vec = sum([skipmodel.wv[word] for word in docs[self.documents[i].id] if  word in skipmodel.wv])
    
    for i in range(len(self.testdocs)):
      self.testdocs[i].w2vec = sum([skipmodel.wv[word] for word in queries[self.testdocs[i].id] if word in skipmodel.wv])
    

In [ ]:
vec = vectorize(documents, testdocs)

In [ ]:
from gensim.parsing.preprocessing import preprocess_string, preprocess_documents 
class MyCorpus:
      """An iterator that yields sentences (lists of str)."""

      def __iter__(self):
        for file in vec.documents:
          yield file.read().lower().split()
        for file in vec.testdocs:
          yield file.read().lower().split()

In [ ]:
sentences = MyCorpus()

In [ ]:
vec.tfidfvec()

In [ ]:
vec.word2vec(sentences)

#tfidf

In [ ]:
%%writefile utils.py



In [ ]:
def perf_measure(y_actual, y_hat):
  """
  calculates TP, FP, TN, FN for actual and predicted y
  """
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score

#classifier for LinearSVC
clf = Pipeline([
    ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1))
      ])

l = {}
for col in columns:
  y = np.array([vec.documents[i].level[col] for i in range(len(vec.documents))])
  clf.fit(vec.tfidfdoc, y)
  ytest = np.array([vec.testdocs[i].level[col] for i in range(len(vec.testdocs))])
  pred = clf.predict(vec.tfidftestdoc)
  l[col] = perf_measure(ytest, pred)
  print(f'{col} accuracy: {accuracy_score(ytest, pred)}')


toxic accuracy: 0.9302260151927225
severe_toxic accuracy: 0.993419612991966
obscene accuracy: 0.9651598987151834
threat accuracy: 0.9968739254118604
insult accuracy: 0.9635812310481728
identity_hate accuracy: 0.9901372346744194


In [ ]:
def micro_precision(dic):
  """
  returns micro_precision for multilabel classification
  """
  num, denom = 0, 0
  for col in dic:
    num += dic[col][0]
    denom += dic[col][0] + dic[col][1]
  return num/denom
  

In [ ]:
def micro_recall(dic):
  """
  returns micro_recall for multilabel classification
  """
  num, denom = 0, 0
  for col in dic:
    num += dic[col][0]
    denom += dic[col][0] + dic[col][3]
  return num/denom

In [ ]:
def micro_f1(micro_p, micro_rec):
  """
  returns micro_f1 for multilabel classification
  """
  return 2*(micro_p*micro_rec)/(micro_p + micro_rec)

In [ ]:
def macro_precision(dic):
  """
  returns macro_precision for multilabel classification
  """
  prec = 0
  for col in dic:
    precision = dic[col][0]/ (dic[col][0] + dic[col][1])
    prec += precision
  return prec/6



In [ ]:
def macro_recall(dic):
  """
  returns macro_recall for multilabel classification
  """
  recall = 0
  for col in dic:
    recall += dic[col][0] / (dic[col][0] + dic[col][3])
  return recall/6

In [ ]:
def macro_f1(macro_p, macro_rec):
  """
  returns macro_f1 for multilabel classification
  """
  return 2*(macro_p*macro_rec)/(macro_p+macro_rec)

In [ ]:
print(f'Micro-Precision for Tfidf: {micro_precision(l)}')
print(f'Macro-Precision for Tfidf: {macro_precision(l)}')
print(f'Micro-recall for Tfidf: {micro_recall(l)}')
print(f'Macro-recall for Tfidf: {macro_recall(l)}')
print(f'Micro-f1 for Tfidf: {micro_f1(micro_precision(l), micro_recall(l))}')
print(f'Macro-f1 for Tfidf: {macro_f1(macro_precision(l), macro_recall(l))}')

Micro-Precision for Tfidf: 0.571709542915585
Macro-Precision for Tfidf: 0.47171411406595704
Micro-recall for Tfidf: 0.5012415505586978
Macro-recall for Tfidf: 0.37082735342713224
Micro-f1 for Tfidf: 0.5341614906832298
Macro-f1 for Tfidf: 0.41523059277729374


#word2vec


In [15]:
dic = {1:'2', 3:'4'}
len(list(dic.items()))

2

In [ ]:
X = np.array([vec.documents[i].w2vec for i in range(len(vec.documents))])

In [ ]:
Xtest = np.array([vec.testdocs[i].w2vec for i in range(len(vec.testdocs))])

In [ ]:
levels = ['toxic', 'severe_toxic', 'obscene', 'threat', 
                                  'insult', 'identity_hate']



In [ ]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
clf = Pipeline([
    ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1))
      ])
l = {}
for col in columns:
  y = np.array([vec.documents[i].level[col] for i in range(len(vec.documents))])
  clf.fit(X, y)
  ytest = np.array([vec.testdocs[i].level[col] for i in range(len(vec.testdocs))])
  pred = clf.predict(Xtest)
  l[col] = perf_measure(ytest, pred)
  print(f'{col} accuracy: {accuracy_score(ytest, pred)}')

toxic accuracy: 0.9179405420613336
severe_toxic accuracy: 0.9880427647003658
obscene accuracy: 0.9534058582637782
threat accuracy: 0.9956860170683672
insult accuracy: 0.9581731220106912
identity_hate accuracy: 0.9886367188721122


In [ ]:
print(f'Micro-Precision for word2vec: {micro_precision(l)}')
print(f'Macro-Precision for word2vec: {macro_precision(l)}')
print(f'Micro-recall for word2vec: {micro_recall(l)}')
print(f'Macro-recall for word2vec: {macro_recall(l)}')
print(f'Micro-f1 for word2vec: {micro_f1(micro_precision(l), micro_recall(l))}')
print(f'Macro-f1 for word2vec: {macro_f1(macro_precision(l), macro_recall(l))}')

Micro-Precision for word2vec: 0.571709542915585
Macro-Precision for word2vec: 0.47171411406595704
Micro-recall for word2vec: 0.5012415505586978
Macro-recall for word2vec: 0.37082735342713224
Micro-f1 for word2vec: 0.5341614906832298
Macro-f1 for word2vec: 0.41523059277729374
